<a href="https://colab.research.google.com/github/cars1015/recsys_MyReserch/blob/main/cos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

# Googleドライブをマウント
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

簡単なコサイン類似度

In [36]:
data="ml-20m"
dir="/content/drive/MyDrive/recommend/" + data + "/pro_sg/"
df=pd.read_csv(dir+"test_tr.csv")

In [39]:
class cos:
    def __init__(self):
      #LabelEncoder()は文字列を数値に変えてくれる
        self.user_enc = LabelEncoder()
        self.item_enc = LabelEncoder()

    def _get_users_and_items(self, df):
        users = self.user_enc.fit_transform(df.loc[:, 'uid'])
        items = self.item_enc.fit_transform(df.loc[:, 'sid'])
        return users, items

    def cos_sim(v1, v2):
        return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    def fit(self, df, implicit=True):
        """
        df: pandas.DataFrame with columns user_id, item_id and (rating)
        lambda_: l2-regularization term
        implicit: if True, ratings are ignored and taken as 1, else normalized ratings are used
        """
        users, items = self._get_users_and_items(df)
        values = np.ones(df.shape[0]) if implicit else df['rating'].to_numpy() / df['rating'].max()

        X = csr_matrix((values, (users, items)))
        self.X = X
    
        X=X.toarray()
        diagIndices=np.diag_indices(X.shape[1])
        B=np.zeros((X.shape[1],X.shape[1]))
        for i in range(X.shape[1]):
          for j in range(X.shape[1]):
            B[i,j]=cos_sim(X[:,i],X[:,j])
        B[diagIndices]=0
        self.B=B


In [ ]:
model=cos()
model.fit(df)

In [ ]:
df_test=pd.read_csv(dir+"test_tr.csv")
users = df_test.loc[:, 'uid']
items = df_test.loc[:, 'sid']

u_enc = LabelEncoder()
users_id = u_enc.fit_transform(users)
items_id = model.item_enc.transform(items)

# 
values = np.ones(df_test.shape[0])
shape = (u_enc.classes_.size, model.item_enc.classes_.size)
X = csr_matrix((values, (users_id, items_id)), shape=shape)
print(X)
print(X.toarray())
#
print(X.T.dot(X).toarray())

pred = X.dot(model.B)

In [20]:
pred=X.dot()
np.savetxt(dir+"pred_cos.txt",pred)

[ 0.          0.21550548  0.01174305 -0.08450545  0.03320105 -0.17986658
 -0.06159746 -0.07742302  0.09398268 -0.00076131  0.0046061   0.02857799
 -0.07517553 -0.03395668 -0.1315152   0.19080944  0.01517345 -0.02964113
 -0.02096879  0.01374048  0.03491639 -0.06736777 -0.00872675  0.15482946
  0.12198146 -0.11254683 -0.05356868 -0.20114499 -0.09942105 -0.23621311]
(30, 30)
